<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/week8_TF_IDF_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week8: This notebook is to get baseline model of TF-IDF.

Situation: English only (=multi-class) genre detection using SMOTE.
Split: StratifiedKfold.


### set up

In [ ]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
DIR = '/content/drive/MyDrive/music4all/'
def get_balanced_accuracy(model, McNemar, is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning):
  test_y = test.map(map_func_only_y)
  y_category=np.zeros((TEST_SIZE, ))
  counter=0
  for label_tensor in test_y.take(len(test_y)):
    y_test = np.argmax(label_tensor, axis=1)
    for label in y_test:
      y_category[counter]=label
      counter+=1

  X_test, y_test = test.map(map_func_only_X), y_category
  y_predict_test = np.asarray(model.predict(X_test))
  y_predict_test = np.argmax(y_predict_test, axis=1)
  print(classification_report(y_test, y_predict_test) )
  print(balanced_accuracy_score(y_test, y_predict_test))

  McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)] = []
  for ground_truh, pred in zip(y_test, y_predict_test):
        if ground_truh==pred:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(True)
        else:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(False)
  with open(DIR+ "ALBERT_large_log.txt", "a") as f:
    print("======================================", file=f)
    print("is_fine_tuning?:", is_fine_tuning, "drop_out_rate: ", drop_out_rate, "learning_rate_transfer_learning: ", learning_rate_transfer_learning,
          "learning_rate_fine_tuning: ", learning_rate_fine_tuning, file=f)
    print(classification_report(y_test, y_predict_test) , file=f)
    print(balanced_accuracy_score(y_test, y_predict_test), file=f)

  return balanced_accuracy_score(y_test, y_predict_test), McNemar

### Data Preparation(Kfold split)

Create dataframe for Gensim

In [ ]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'df_genre_by_lang_full.csv')
df_genre_by_lang

,Unnamed: 0,id,genres,lang,lyric,number_of_line
0,0,0009fFIM1eYThaPg,pop,en,a sunny day so I got nowhere to hide Not a clo...,91
1,1,00P2bHdWFkghmDqz,soul,en,Tell me a tale that always was Sing me a song ...,36
2,2,00b6fV3nx5z2b8Ls,pop,en,A buh A buh You went to school to learn girl T...,74
3,3,013QDoTqbexEwkHr,pop,en,like a conversation where stops to breathe Is ...,20
4,4,01EKNot8qVgZpKM7,rock,en,Say the words I cannot say Say them on another...,31
...,...,...,...,...,...,...
13535,13535,zzT504Z94j1IAuc3,indie rock,en,think what afraid of come in you know been mad...,18
13536,13536,zzgS4ZqyswamEWNj,pop,en,Oh yeah yeah Last night I took a walk in the s...,75
13537,13537,zzx8CWdM7qkxKQpC,indie rock,en,Innocence it come easy in a sense it never wil...,34
13538,13538,zzz0n04uuTUA7fNh,pop,en,Girl you know how I feel I really Since you be...,65


In [ ]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [ ]:
data = load_data(df_genre_by_lang["lyric"], df_genre_by_lang["genres"])

In [ ]:
data

,texts,labels
0,"[a, sunny, day, so, I, got, nowhere, to, hide,...",pop
1,"[Tell, me, a, tale, that, always, was, Sing, m...",soul
2,"[A, buh, A, buh, You, went, to, school, to, le...",pop
3,"[like, a, conversation, where, stops, to, brea...",pop
4,"[Say, the, words, I, cannot, say, Say, them, o...",rock
...,...,...
13535,"[think, what, afraid, of, come, in, you, know,...",indie rock
13536,"[Oh, yeah, yeah, Last, night, I, took, a, walk...",pop
13537,"[Innocence, it, come, easy, in, a, sense, it, ...",indie rock
13538,"[Girl, you, know, how, I, feel, I, really, Sin...",pop


In [ ]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [ ]:
type(X)

pandas.core.series.Series

In [ ]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [    0     1     3 ... 13537 13538 13539] TEST: [    2     4     5 ... 13526 13532 13535]
TRAIN: [    0     2     4 ... 13535 13536 13539] TEST: [    1     3     7 ... 13530 13537 13538]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [    8    14    22 ... 13521 13531 13536]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [   10    12    15 ... 13523 13525 13534]
TRAIN: [    1     2     3 ... 13536 13537 13538] TEST: [    0     6    11 ... 13529 13533 13539]


In [ ]:
len(splits)

5

In [ ]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(10832,)
(10832,)
(2708,)
(2708,)


In [ ]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [ ]:
split0['y_train']

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

###  Train and Test flow

Step1: Kfold Stratified Split

Step2: For X_train, get Bag-of-Words feature using TF-IDF.

Step3: clf.fit(X_train, y_train)

Step4: transform X_test['lyric'] to TF-IDF

Step5: Predict by, clf_predict = clf.predict(X_test_tf-idf)

Step6: Compare (clf_predict, y_test)

### Multinomial Nayve Bayes trained on TF-IDF embedding. Test KFold mean 

In [ ]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

for i in range(5):
  print("Split: ", i)
  split0 = splits[i]
  for i, token_list in split0['X_train'].items():
    split0['X_train'][i] =  ' '.join(token_list)
  for i, token_list in split0['X_test'].items():
    split0['X_test'][i] =  ' '.join(token_list)

  tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')
  X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train']).toarray()
  X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']).toarray()

  df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                                ,index = split0['X_train'].index
                              , columns=tfidf_vectorizer.get_feature_names())
  
  X_resampled, y_resampled = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train']) 
  print(sorted(Counter(y_resampled).items()))
  clf_mnb_SMOTE = MultinomialNB().fit(X_resampled, y_resampled)

  predicted_mnb_SMOTE = clf_mnb_SMOTE.predict(X_test_tfidf)
  print(classification_report(split0['y_test'], predicted_mnb_SMOTE) )
  print(balanced_accuracy_score(split0['y_test'], predicted_mnb_SMOTE))

  del(split0)
  del(df_X_train_tfidf)
  del(X_resampled)
  del(y_resampled)

Split:  0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 4143), (1, 4143), (2, 4143), (3, 4143), (4, 4143), (5, 4143), (6, 4143), (7, 4143), (8, 4143), (9, 4143)]
              precision    recall  f1-score   support

           0       0.20      0.11      0.14       190
           1       0.17      0.33      0.22       173
           2       0.09      0.16      0.12       135
           3       0.25      0.17      0.20       216
           4       0.58      0.34      0.43      1036
           5       0.25      0.20      0.22        76
           6       0.59      0.79      0.68       195
           7       0.20      0.14      0.17       290
           8       0.14      0.18      0.16       139
           9       0.21      0.47      0.29       258

    accuracy                           0.31      2708
   macro avg       0.27      0.29      0.26      2708
weighted avg       0.37      0.31      0.32      2708

0.2882247048904138
Split:  1


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 4143), (1, 4143), (2, 4143), (3, 4143), (4, 4143), (5, 4143), (6, 4143), (7, 4143), (8, 4143), (9, 4143)]
              precision    recall  f1-score   support

           0       0.25      0.08      0.13       191
           1       0.17      0.39      0.24       173
           2       0.08      0.11      0.09       134
           3       0.16      0.12      0.14       216
           4       0.60      0.32      0.42      1036
           5       0.28      0.24      0.26        76
           6       0.58      0.88      0.70       196
           7       0.18      0.15      0.16       289
           8       0.10      0.13      0.11       139
           9       0.25      0.55      0.34       258

    accuracy                           0.31      2708
   macro avg       0.27      0.30      0.26      2708
weighted avg       0.37      0.31      0.31      2708

0.2977563709936377
Split:  2


...

[(0, 4143), (1, 4143), (2, 4143), (3, 4143), (4, 4143), (5, 4143), (6, 4143), (7, 4143), (8, 4143), (9, 4143)]
              precision    recall  f1-score   support

           0       0.19      0.05      0.08       191
           1       0.18      0.38      0.24       172
           2       0.09      0.13      0.10       134
           3       0.20      0.13      0.16       216
           4       0.60      0.37      0.46      1036
           5       0.24      0.19      0.21        77
           6       0.67      0.84      0.74       196
           7       0.17      0.13      0.15       290
           8       0.11      0.16      0.13       139
           9       0.22      0.50      0.31       257

    accuracy                           0.32      2708
   macro avg       0.27      0.29      0.26      2708
weighted avg       0.37      0.32      0.33      2708

0.2885083316539593
Split:  3


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 4143), (1, 4143), (2, 4143), (3, 4143), (4, 4143), (5, 4143), (6, 4143), (7, 4143), (8, 4143), (9, 4143)]
              precision    recall  f1-score   support

           0       0.21      0.06      0.09       191
           1       0.18      0.41      0.25       172
           2       0.07      0.11      0.09       134
           3       0.18      0.12      0.15       216
           4       0.56      0.34      0.42      1036
           5       0.26      0.19      0.22        77
           6       0.63      0.88      0.73       196
           7       0.17      0.11      0.13       290
           8       0.09      0.11      0.10       139
           9       0.23      0.53      0.32       257

    accuracy                           0.31      2708
   macro avg       0.26      0.29      0.25      2708
weighted avg       0.36      0.31      0.31      2708

0.2871302077055935
Split:  4


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 4144), (1, 4144), (2, 4144), (3, 4144), (4, 4144), (5, 4144), (6, 4144), (7, 4144), (8, 4144), (9, 4144)]
              precision    recall  f1-score   support

           0       0.17      0.07      0.10       190
           1       0.18      0.33      0.23       173
           2       0.13      0.16      0.14       135
           3       0.21      0.14      0.17       217
           4       0.59      0.38      0.46      1035
           5       0.38      0.33      0.35        76
           6       0.65      0.85      0.74       195
           7       0.19      0.17      0.18       290
           8       0.14      0.19      0.16       139
           9       0.24      0.53      0.33       258

    accuracy                           0.34      2708
   macro avg       0.29      0.31      0.29      2708
weighted avg       0.38      0.34      0.34      2708

0.3146080778345702


In [ ]:
balanced_accuracy=[0.2882247048904138, 0.2977563709936377, 0.2885083316539593, 0.2871302077055935, 0.3146080778345702]

In [ ]:
sum(balanced_accuracy)/5

0.2952455386156349

### 2.Support Vector Machine use kernel='rbf' gamma = 'auto', C=10.0

Fold0

In [ ]:
from sklearn import svm
from imblearn.over_sampling import SMOTE
clf_svm = svm.SVC(C=10.0, kernel='rbf', degree=10, gamma='auto')
X_res, y_res = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train'])
print(Counter(y_res))

Counter({4: 4143, 9: 4143, 6: 4143, 2: 4143, 0: 4143, 7: 4143, 1: 4143, 5: 4143, 8: 4143, 3: 4143})


In [ ]:
clf_svm.fit(X_res, y_res)

SVC(C=10.0, degree=10, gamma='auto')

Took 34 min.

In [ ]:
X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']['lyric']).toarray()

KeyError: ignored

In [ ]:
X_test_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
split0['X_test']

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score,classification_report

predicted_svm_SMOTE = clf_svm.predict(X_test_tfidf)
print(metrics.classification_report(split0['y_test'], predicted_svm_SMOTE) )
balanced_accuracy_score(split0['y_test'], predicted_svm_SMOTE)

In [ ]:
DIR = '/content/drive/MyDrive/music4all/'
with open(DIR+ "TF-IDF_svm_log.txt", "a") as f:
  print("Fold0", file=f)
  print(metrics.classification_report(split0['y_test'], predicted_svm_SMOTE) , file=f)
  print(balanced_accuracy_score(split0['y_test'], predicted_svm_SMOTE), file=f)


In [ ]:
balanced_accuracy=[]
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], predicted_svm_SMOTE))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(split0['y_test'], predicted_svm_SMOTE
                                        , cmap=plt.cm.Blues)
plt.figure(figsize=(10, 10))
plt.show()

*italicized text*### Fold3 (split[2])

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
balanced_accuracy=[]

Unit test

In [ ]:
for i, token_list in split0['X_train'].items():
    lyric=""
    for word in token_list:
      if wordnet.synsets(word):
        word = " " +word
        #print(word)
        lyric +=word
    split0['X_train'][i] =  lyric
for i, token_list in split0['X_test'].items():
  lyric=""
  for word in token_list:
    if wordnet.synsets(word):
      word = " " +word
      #print(word)
      lyric +=word
  split0['X_test'][i] =  lyric
split0['X_train'][0:20]
split0['X_test'][0:20]

0      a sunny day so I got nowhere hide Not a cloud...
1      Tell me a tale always was Sing me a song alwa...
3      like a conversation stops breathe Is it imagi...
6      Locked up tight Like I never feel again Stuck...
7      in crib about leer jets coupes way salt sell ...
8      Weekends in bed no scramble eggs or bacon I j...
10     every breath I breathe making history name on...
11     is I know canopy loss home Is a far cry while...
12     Ooh I see over there be shy come here got thi...
13     a boy lost way looking someone play a girl in...
14     Can I be honest Can I tell I never did mind s...
15     Ooh tell me I do supposed be I drop it now Bu...
16     dog eat just waking up dove is in dungeon haw...
17     never met type ignored me alright Never be fr...
18     I wish I believe in bigger More trees winds o...
19     Yeah yeah yeah yeah Yeah yeah yeah yeah Yeah ...
21     walk room So perfect but unaware Making me st...
22     feet are cracked As break back In all yes

In [ ]:
split0['X_train'][0:20]
split0['X_test'][0:20]

2     [A, buh, A, buh, You, went, to, school, to, le...
4     [Say, the, words, I, cannot, say, Say, them, o...
5     [I, was, alone, I, was, made, of, stone, You, ...
9     [Again, the, burden, of, losing, rests, upon, ...
20    [only, been, three, weeks, And, a, bag, of, sp...
23    [Björk, My, blood, flows, in, me, My, red, jui...
24    [oh, no, no, no, no, no, no, I, know, That, ju...
34    [When, I, think, of, you, in, the, city, The, ...
45    [How, did, it, get, to, I, wish, that, I, coul...
55    [And, irony, is, okay, I, suppose, culture, is...
56    [Boy, brought, power, To, Hear, me, call, To, ...
57    [I, wanted, you, to, know, that, I, love, the,...
60    [Now, floating, like, two, feathers, In, the, ...
66    [Oh, oh, yeah, I, get, loud, when, you, put, t...
67    [Sometimes, I, get, tired, of, this, attitude,...
68    [not, twenty, and, detached, now, Way, too, ea...
72    [the, look, in, your, eyes, that, told, me, th...
73    [Da, da, da, da, The, smell, of, your, ski

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train'][0:20]).toarray()
df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                              ,index = split0['X_train'][0:20].index
                            , columns=tfidf_vectorizer.get_feature_names())
X_test_tfidf = tfidf_vectorizer.transform(split0['X_test'][0:20]).toarray()
print(df_X_train_tfidf.shape)

#fit
clf_svm = svm.SVC(C=1.0, kernel='rbf', degree=10, gamma=1.0)
clf_svm.fit(df_X_train_tfidf.to_numpy()[0:20], split0['y_train'][0:20])

(20, 723)


SVC(degree=10, gamma=1.0)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
predicted_svm_SMOTE = clf_svm.predict(X_test_tfidf[0:20])
print(classification_report(split0['y_test'][0:20], predicted_svm_SMOTE) )
balanced_acc =balanced_accuracy_score(split0['y_test'][0:20], predicted_svm_SMOTE)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3
           4       0.30      1.00      0.46         6
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         2

    accuracy                           0.30        20
   macro avg       0.04      0.12      0.06        20
weighted avg       0.09      0.30      0.14        20



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
balanced_acc

0.125

### Fold0

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from imblearn.over_sampling import SMOTE
from nltk.corpus import wordnet
DIR = '/content/drive/MyDrive/music4all/'

for i in range(0,1):
  split0 = splits[i]
  for i, token_list in split0['X_train'].items():
    lyric=""
    for word in token_list:
      if wordnet.synsets(word):
        word = " " +word
        #print(word)
        lyric +=word
    split0['X_train'][i] =  lyric
  for i, token_list in split0['X_test'].items():
    lyric=""
    for word in token_list:
      if wordnet.synsets(word):
        word = " " +word
        #print(word)
        lyric +=word
    split0['X_test'][i] =  lyric
  
  tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')
  X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train']).toarray()
  df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                                ,index = split0['X_train'].index
                              , columns=tfidf_vectorizer.get_feature_names())
  X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']).toarray()
  print(df_X_train_tfidf.shape)

  #fit
  clf_svm = svm.SVC(C=1.0, kernel='rbf', degree=10, gamma=1.0)
  X_res, y_res = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train'])
  print(Counter(y_res))
  clf_svm.fit(X_res, y_res)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(10832, 21795)
Counter({4: 4143, 9: 4143, 6: 4143, 2: 4143, 0: 4143, 7: 4143, 1: 4143, 5: 4143, 8: 4143, 3: 4143})


In [ ]:
predicted_svm_SMOTE = clf_svm.predict(X_test_tfidf)
print(classification_report(split0['y_test'], predicted_svm_SMOTE) )
balanced_acc =balanced_accuracy_score(split0['y_test'], predicted_svm_SMOTE)
print(balanced_acc)
balanced_accuracy.append(balanced_acc )

with open(DIR+ "TF-IDF_svm_log.txt", "a") as f:
  print("Fold:", i, file=f)
  print(classification_report(split0['y_test'], predicted_svm_SMOTE) , file=f)
  print(balanced_acc, file=f)

              precision    recall  f1-score   support

           0       0.26      0.03      0.05       190
           1       0.33      0.05      0.09       173
           2       0.57      0.03      0.06       135
           3       0.44      0.02      0.04       216
           4       0.41      0.94      0.57      1036
           5       0.43      0.04      0.07        76
           6       0.90      0.59      0.72       195
           7       0.31      0.04      0.07       290
           8       0.30      0.02      0.04       139
           9       0.30      0.07      0.11       258

    accuracy                           0.42      2708
   macro avg       0.43      0.18      0.18      2708
weighted avg       0.41      0.42      0.31      2708

0.18302532232708063


### Fold1-4

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from imblearn.over_sampling import SMOTE
DIR = '/content/drive/MyDrive/music4all/'

for i in range(1,5):
  split0 = splits[i]
  for i, token_list in split0['X_train'].items():
    split0['X_train'][i] =  ' '.join(token_list)
  for i, token_list in split0['X_test'].items():
    split0['X_test'][i] =  ' '.join(token_list)
  
  tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')
  X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train']).toarray()
  df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                                ,index = split0['X_train'].index
                              , columns=tfidf_vectorizer.get_feature_names())
  X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']).toarray()

  #fit
  clf_svm = svm.SVC(C=10.0, kernel='rbf', degree=10, gamma='auto')
  X_res, y_res = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train'])
  clf_svm.fit(X_res, y_res)

  predicted_svm_SMOTE = clf_svm.predict(X_test_tfidf)
  print(metrics.classification_report(split0['y_test'], predicted_svm_SMOTE) )
  balanced_acc =balanced_accuracy_score(split0['y_test'], predicted_svm_SMOTE)
  print(balanced_acc)
  balanced_accuracy.append(balanced_acc )

  with open(DIR+ "TF-IDF_svm_log.txt", "a") as f:
    print("Fold:", i, file=f)
    print(metrics.classification_report(split0['y_test'], predicted_svm_SMOTE) , file=f)
    print(balanced_acc, file=f)

In [ ]:
sum(balanced_accuracy)/5